Εισαγωγή των απαραίτητων βιβλιοθηκών.

In [1]:
import cv2
import os
from sklearn.cluster import MiniBatchKMeans #KMeans
from sklearn.cluster import MeanShift, estimate_bandwidth,AffinityPropagation

import numpy as np
#import secondary functions that will be used very frequent
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd 

Δημιουργούμε ένα dataframe στο οποίο θα αποθηκεύσουμε τα αποτελέσματα μας.

In [2]:
columns = ['FeatureExtraction', 'Clustering Detection','Train Data ratio',
           'Classifier Used','Accuracy(tr)', 'Precision(tr)',
           'Recal(tr)', 'F1 score (tr)','Accuracy (te)', 'Precision (te)',
           'Recal(te)', 'F1 score (te)']

results_DF = pd.DataFrame(columns=columns)


Συνάρτηση με την οποία διαβάζουμε όλες τις εικόνες και δημιουργούμε ένα dictionary όπου περιέχει όλες τις εικόνες ανά κατηγορία.

In [3]:
# return a dictionary that holds all images category by category.
def load_images_from_folder(folder, inputImageSize ):
    images = {}
    for filename in os.listdir(folder):
        category = []
        path = folder + "/" + filename
        for name in os.listdir(path):
            img = cv2.imread(path + "/" + name)
            #print(' .. parsing image', cat)
            if img is not None:
                # grayscale it
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                #resize it, if necessary
                img = cv2.resize(img, (inputImageSize[0], inputImageSize[1]))

                category.append(img)
        images[filename] = category
        print(' . Finished parsing images. What is next?')
    return images

In [4]:
#define a fixed image size to work with
inputImageSize = [200, 200, 3] 

In [5]:
TrainImagesFilePath ='Dataset/Train'
TestImagesFilePath = 'Dataset/Test'

Διαβάζουμε τις εικόνες προς εκπαίδευση.

In [6]:
#load the train images
trainImages = load_images_from_folder(TrainImagesFilePath, inputImageSize)  # take all images category by category for train set
trainImages.__len__()

 . Finished parsing images. What is next?
 . Finished parsing images. What is next?


2

In [7]:
# Creates descriptors using an approach of your choise. e.g. ORB, SIFT, SURF, FREAK, MOPS, etc
# Takes first parameter that is images dictionary
# Takes second parameter that is the Detector
# Return an array whose first index holds the decriptor_list without an order
# And the second index holds the sift_vectors dictionary which holds the descriptors but this is seperated class by class
def detector_features(images, detectorToUse):
    print(' . start detecting points and calculating features for a given image set')
    detector_vectors = {}
    descriptor_list = []
    
    for nameOfCategory, availableImages in images.items():
        features = []
        for img in availableImages: # reminder: val
            kp, des = detectorToUse.detectAndCompute(img, None)

            descriptor_list.extend(des)
            features.append(des)
        detector_vectors[nameOfCategory] = features
        print(' . finished detecting points and calculating features for a given image set')
    return [descriptor_list, detector_vectors] # be aware of the []! this is ONE output as a list


In [8]:
trainDataFeatures_SIFT = detector_features(trainImages, cv2.xfeatures2d.SIFT_create())

 . start detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set


In [9]:
trainDataFeatures_ORB = detector_features(trainImages, cv2.ORB_create())


 . start detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set


In [10]:
trainDataFeatures_SURF = detector_features(trainImages, cv2.xfeatures2d.SURF_create(400))

 . start detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set


In [11]:
# Takes the descriptor list which is unordered one
TrainDescriptorList_SIFT = trainDataFeatures_SIFT[0]
TrainDescriptorList_ORB = trainDataFeatures_ORB[0]
TrainDescriptorList_SURF = trainDataFeatures_SURF[0]

In [12]:
TrainDescriptorList_SIFT.__len__()

6784

In [13]:
TrainDescriptorList_SURF.__len__()

4355

In [14]:
# A k-means clustering algorithm who takes 2 parameter which is number
# of cluster(k) and the other is descriptors list(unordered 1d array)
# Returns an array that holds central points.
def kmeansVisualWordsCreation(k, descriptor_list):
    print(' . calculating central points for the existing feature values.')
    #kmeansModel = KMeans(n_clusters = k, n_init=10)
    batchSize = np.ceil(descriptor_list.__len__()/50).astype('int')
    kmeansModel = MiniBatchKMeans(n_clusters=k, batch_size=batchSize, verbose=0)
    kmeansModel.fit(descriptor_list)
    visualWords = kmeansModel.cluster_centers_ # a.k.a. centers of reference
    print(' . done calculating central points for the given feature set.Centers are ', len(kmeansModel.cluster_centers_))
    return visualWords, kmeansModel

In [15]:
# A mean shift clustering algorithm who takes 1 parameter which is 
# descriptors list(unordered 1d array)
# Returns an array that holds central points.
def run_mean_shift(descriptor_list, bbandwidth):
    
    # The following bandwidth can be automatically detected using
    #bandwidth = estimate_bandwidth(descriptor_list)
    #print(bandwidth)
    ms = MeanShift(bandwidth = bbandwidth).fit(descriptor_list)
    
    visualWords = ms.cluster_centers_
    print(' . done calculating central points for the given feature set.')
    print('Centers is', len(ms.cluster_centers_))
    return visualWords, ms


In [16]:
#create the central points for the histograms using mean shift.
visualWords_SIFT_ms, TrainedMsModel_SIFT = run_mean_shift(TrainDescriptorList_SIFT
                                                         ,300)

 . done calculating central points for the given feature set.
Centers is 2731


In [17]:
visualWords_ORB_ms, TrainedMsModel_ORB = run_mean_shift(TrainDescriptorList_ORB
                                                       ,300)

 . done calculating central points for the given feature set.
Centers is 3329


In [27]:
visualWords_SURF_ms, TrainedMsModel_SURF = run_mean_shift(TrainDescriptorList_SURF
                                                         ,0.4)

 . done calculating central points for the given feature set.
Centers is 115


In [28]:
#create the central points for the histograms using k means.
#here we use a rule of the thumb to create the expected number of cluster centers
numberOfClasses = trainImages.__len__() #retrieve num of classes from dictionary
possibleNumOfCentersToUse = 10 * numberOfClasses

visualWords_SIFT, TrainedKmeansModel_SIFT = kmeansVisualWordsCreation(possibleNumOfCentersToUse,
                                                            TrainDescriptorList_SIFT)

visualWords_ORB, TrainedKmeansModel_ORB = kmeansVisualWordsCreation(possibleNumOfCentersToUse,
                                                            TrainDescriptorList_ORB)

visualWords_SURF, TrainedKmeansModel_SURF = kmeansVisualWordsCreation(possibleNumOfCentersToUse,
                                                            TrainDescriptorList_SURF)


 . calculating central points for the existing feature values.
 . done calculating central points for the given feature set.Centers are  20
 . calculating central points for the existing feature values.
 . done calculating central points for the given feature set.Centers are  20
 . calculating central points for the existing feature values.
 . done calculating central points for the given feature set.Centers are  20


Το επόμενο βήμα είναι να δημιουργήσουμε τα ιστογράμματα.

In [29]:
# Takes the sift feature values that is seperated class by class for train data,
#we need this to calculate the histograms
trainBoVWFeatureVals_SIFT = trainDataFeatures_SIFT[1]

trainBoVWFeatureVals_ORB = trainDataFeatures_ORB[1]

trainBoVWFeatureVals_SURF = trainDataFeatures_SURF[1]

In [30]:
def mapFeatureValsToHistogram (DataFeaturesByClass, visualWords, MODEL):
    #depenting on the approach you may not need to use all inputs
    histogramsList = []
    targetClassList = []
    numberOfBinsPerHistogram = visualWords.shape[0]

    for categoryIdx, featureValues in DataFeaturesByClass.items():
        for tmpImageFeatures in featureValues: #yes, we check one by one the values in each image for all images
            tmpImageHistogram = np.zeros(numberOfBinsPerHistogram)
            tmpIdx = list(MODEL.predict(tmpImageFeatures))
            clustervalue, visualWordMatchCounts = np.unique(tmpIdx, return_counts=True)
            tmpImageHistogram[clustervalue] = visualWordMatchCounts
            # do not forget to normalize the histogram values
            numberOfDetectedPointsInThisImage = tmpIdx.__len__()
            tmpImageHistogram = tmpImageHistogram/numberOfDetectedPointsInThisImage

            #now update the input and output coresponding lists
            histogramsList.append(tmpImageHistogram)
            targetClassList.append(categoryIdx)

    return histogramsList, targetClassList

K means

In [31]:
#create the train input train output format
trainHistogramsList_SIFT, trainTargetsList_SIFT = mapFeatureValsToHistogram(
    trainBoVWFeatureVals_SIFT, visualWords_SIFT, TrainedKmeansModel_SIFT)


In [32]:
#create the train input train output format
trainHistogramsList_ORB, trainTargetsList_ORB = mapFeatureValsToHistogram(
    trainBoVWFeatureVals_ORB, visualWords_ORB, TrainedKmeansModel_ORB)


In [33]:
#create the train input train output format
trainHistogramsList_SURF, trainTargetsList_SURF = mapFeatureValsToHistogram(
    trainBoVWFeatureVals_SURF, visualWords_SURF, TrainedKmeansModel_SURF)

Mean shift

In [34]:
#create the train input train output format
trainHistogramsList_SIFT_ms, trainTargetsList_SIFT_ms = mapFeatureValsToHistogram(
    trainBoVWFeatureVals_SIFT, visualWords_SIFT_ms, TrainedMsModel_SIFT)

In [35]:
#create the train input train output format
trainHistogramsList_ORB_ms, trainTargetsList_ORB_ms = mapFeatureValsToHistogram(
    trainBoVWFeatureVals_ORB, visualWords_ORB_ms, TrainedMsModel_ORB)

In [36]:
#create the train input train output format
trainHistogramsList_SURF_ms, trainTargetsList_SURF_ms = mapFeatureValsToHistogram(
    trainBoVWFeatureVals_SURF, visualWords_SURF_ms, TrainedMsModel_SURF)

In [37]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [38]:
X_train_SIFT = np.stack(trainHistogramsList_SIFT, axis= 0)
# Convert Categorical Data For Scikit-Learn

labelEncoder_SIFT = preprocessing.LabelEncoder()
labelEncoder_SIFT.fit(trainTargetsList_SIFT)
#convert the categories from strings to names
y_train_SIFT = labelEncoder_SIFT.transform(trainTargetsList_SIFT)

knn_SIFT = KNeighborsClassifier()
knn_SIFT.fit(X_train_SIFT, y_train_SIFT)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn_SIFT.score(X_train_SIFT, y_train_SIFT)))

clf_SIFT = DecisionTreeClassifier().fit(X_train_SIFT, y_train_SIFT)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf_SIFT.score(X_train_SIFT, y_train_SIFT)))

svm_SIFT = SVC()
svm_SIFT.fit(X_train_SIFT, y_train_SIFT)
print('Accuracy of svm_SIFT classifier on training set: {:.2f}'.format(svm_SIFT.score(X_train_SIFT, y_train_SIFT)))    


Accuracy of K-NN classifier on training set: 0.57
Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of svm_SIFT classifier on training set: 0.79


In [39]:
X_train_ORB = np.stack(trainHistogramsList_ORB, axis= 0)
# Convert Categorical Data For Scikit-Learn

labelEncoder_ORB = preprocessing.LabelEncoder()
labelEncoder_ORB.fit(trainTargetsList_ORB)
#convert the categories from strings to names
y_train_ORB = labelEncoder_ORB.transform(trainTargetsList_ORB)

knn_ORB = KNeighborsClassifier()
knn_ORB.fit(X_train_ORB, y_train_ORB)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn_ORB.score(X_train_ORB, y_train_ORB)))

clf_ORB = DecisionTreeClassifier().fit(X_train_ORB, y_train_ORB)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf_ORB.score(X_train_ORB, y_train_ORB)))

svm_ORB = SVC()
svm_ORB.fit(X_train_ORB, y_train_ORB)
print('Accuracy of svm_ORB classifier on training set: {:.2f}'.format(svm_ORB.score(X_train_ORB, y_train_ORB)))    


Accuracy of K-NN classifier on training set: 0.82
Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of svm_ORB classifier on training set: 0.89


In [40]:
X_train_SURF = np.stack(trainHistogramsList_SURF, axis= 0)
# Convert Categorical Data For Scikit-Learn

labelEncoder_SURF = preprocessing.LabelEncoder()
labelEncoder_SURF.fit(trainTargetsList_SURF)
#convert the categories from strings to names
y_train_SURF = labelEncoder_SURF.transform(trainTargetsList_SURF)

knn_SURF = KNeighborsClassifier()
knn_SURF.fit(X_train_SURF, y_train_SURF)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn_SURF.score(X_train_SURF, y_train_SURF)))

clf_SURF = DecisionTreeClassifier().fit(X_train_SURF, y_train_SURF)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf_SURF.score(X_train_SURF, y_train_SURF)))

svm_SURF = SVC()
svm_SURF.fit(X_train_SURF, y_train_SURF)
print('Accuracy of svm_SURF classifier on training set: {:.2f}'.format(svm_SURF.score(X_train_SURF, y_train_SURF)))    


Accuracy of K-NN classifier on training set: 0.79
Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of svm_SURF classifier on training set: 0.89


Mean shift

In [41]:
X_train_SIFT_ms = np.stack(trainHistogramsList_SIFT_ms, axis= 0)
# Convert Categorical Data For Scikit-Learn

labelEncoder_SIFT_ms = preprocessing.LabelEncoder()
labelEncoder_SIFT_ms.fit(trainTargetsList_SIFT_ms)
#convert the categories from strings to names
y_train_SIFT_ms = labelEncoder_SIFT_ms.transform(trainTargetsList_SIFT_ms)

knn_SIFT_ms = KNeighborsClassifier()
knn_SIFT_ms.fit(X_train_SIFT_ms, y_train_SIFT_ms)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn_SIFT_ms.score(X_train_SIFT_ms, y_train_SIFT_ms)))

clf_SIFT_ms = DecisionTreeClassifier().fit(X_train_SIFT_ms, y_train_SIFT_ms)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf_SIFT_ms.score(X_train_SIFT_ms, y_train_SIFT_ms)))

svm_SIFT_ms = SVC()
svm_SIFT_ms.fit(X_train_SIFT_ms, y_train_SIFT_ms)
print('Accuracy of svm_SIFT_ms classifier on training set: {:.2f}'.format(svm_SIFT_ms.score(X_train_SIFT_ms, y_train_SIFT_ms)))    


Accuracy of K-NN classifier on training set: 0.68
Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of svm_SIFT_ms classifier on training set: 0.68


In [42]:
X_train_ORB_ms = np.stack(trainHistogramsList_ORB_ms, axis= 0)
# Convert Categorical Data For Scikit-Learn

labelEncoder_ORB_ms = preprocessing.LabelEncoder()
labelEncoder_ORB_ms.fit(trainTargetsList_ORB_ms)
#convert the categories from strings to names
y_train_ORB_ms = labelEncoder_ORB_ms.transform(trainTargetsList_ORB_ms)

knn_ORB_ms = KNeighborsClassifier()
knn_ORB_ms.fit(X_train_ORB_ms, y_train_ORB_ms)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn_ORB_ms.score(X_train_ORB_ms, y_train_ORB_ms)))

clf_ORB_ms = DecisionTreeClassifier().fit(X_train_ORB_ms, y_train_ORB_ms)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf_ORB_ms.score(X_train_ORB_ms, y_train_ORB_ms)))

svm_ORB_ms = SVC()
svm_ORB_ms.fit(X_train_ORB_ms, y_train_ORB_ms)
print('Accuracy of svm_ORB_ms classifier on training set: {:.2f}'.format(svm_ORB_ms.score(X_train_ORB_ms, y_train_ORB_ms)))    


Accuracy of K-NN classifier on training set: 0.75
Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of svm_ORB_ms classifier on training set: 0.86


In [43]:
X_train_SURF_ms = np.stack(trainHistogramsList_SURF_ms, axis= 0)
# Convert Categorical Data For Scikit-Learn

labelEncoder_SURF_ms = preprocessing.LabelEncoder()
labelEncoder_SURF_ms.fit(trainTargetsList_SURF_ms)
#convert the categories from strings to names
y_train_SURF_ms = labelEncoder_SURF_ms.transform(trainTargetsList_SURF_ms)

knn_SURF_ms = KNeighborsClassifier()
knn_SURF_ms.fit(X_train_SURF_ms, y_train_SURF_ms)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn_SURF_ms.score(X_train_SURF_ms, y_train_SURF_ms)))

clf_SURF_ms = DecisionTreeClassifier().fit(X_train_SURF_ms, y_train_SURF_ms)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf_SURF_ms.score(X_train_SURF_ms, y_train_SURF_ms)))

svm_SURF_ms = SVC()
svm_SURF_ms.fit(X_train_SURF_ms, y_train_SURF_ms)
print('Accuracy of svm_SURF_ms classifier on training set: {:.2f}'.format(svm_SURF_ms.score(X_train_SURF_ms, y_train_SURF_ms)))


Accuracy of K-NN classifier on training set: 0.71
Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of svm_SURF_ms classifier on training set: 0.75


In [44]:
#clear some space
del  trainImages, trainBoVWFeatureVals_SIFT,trainBoVWFeatureVals_SURF
del trainBoVWFeatureVals_ORB, trainDataFeatures_SIFT,trainDataFeatures_ORB
del trainDataFeatures_SURF, TrainDescriptorList_SIFT,TrainDescriptorList_ORB
del TrainDescriptorList_SURF

In [45]:
#load the train images
testImages = load_images_from_folder(TestImagesFilePath, inputImageSize)  # take all images category by category for train set


 . Finished parsing images. What is next?
 . Finished parsing images. What is next?


In [46]:
testDataFeatures_SIFT = detector_features(testImages, cv2.xfeatures2d.SIFT_create())
testDataFeatures_ORB = detector_features(testImages, cv2.ORB_create())
testDataFeatures_SURF = detector_features(testImages, cv2.xfeatures2d.SURF_create(400))

 . start detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set
 . start detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set
 . start detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set
 . finished detecting points and calculating features for a given image set


In [47]:
# Takes the descriptor list which is unordered one
testDescriptorList_SIFT = testDataFeatures_SIFT[0]
testDescriptorList_ORB = testDataFeatures_ORB[0]
testDescriptorList_SURF = testDataFeatures_SURF[0]

In [48]:
# Takes the sift feature values that is seperated class by class for train data, we need this to calculate the histograms
testBoVWFeatureVals_SIFT = testDataFeatures_SIFT[1]

testBoVWFeatureVals_ORB = testDataFeatures_ORB[1]

testBoVWFeatureVals_SURF = testDataFeatures_SURF[1]

In [49]:
def toExcel (FeatureExtraction, clusteringTech ,
             dataRatio, Classifier,  X_train, X_test, 
             y_train,y_test,y_pred_train,  y_pred_test):
    
    #calculate the scores
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)
    pre_train = precision_score(y_train, y_pred_train, average='macro', zero_division=1)
    pre_test = precision_score(y_test, y_pred_test, average='macro', zero_division = 1)
    rec_train = recall_score(y_train, y_pred_train, average='macro')
    rec_test = recall_score(y_test, y_pred_test, average='macro')
    f1_train = f1_score(y_train, y_pred_train, average='macro')
    f1_test = f1_score(y_test, y_pred_test, average='macro')

    
    #df
    results_DF.loc[len(results_DF)] = [FeatureExtraction ] + [clusteringTech] \
                                      +  [dataRatio] + [Classifier] + [acc_train]\
                                      + [pre_train]  + [rec_train] + [f1_train] \
                                      + [acc_test]   + [pre_test] + [rec_test] + [f1_test]

K means

In [50]:
#create the test input / test output format
#create the train input train output format
testHistogramsList_SIFT, testTargetsList_SIFT = mapFeatureValsToHistogram(
    testBoVWFeatureVals_SIFT, visualWords_SIFT, TrainedKmeansModel_SIFT)
X_test_SIFT = np.array(testHistogramsList_SIFT)

y_test_SIFT = labelEncoder_SIFT.transform(testTargetsList_SIFT)

y_pred_train_SIFT = clf_SIFT.predict(X_train_SIFT )
y_pred_test_SIFT = clf_SIFT.predict(X_test_SIFT)
toExcel('SIFT', 'KNN', '60:40', 'clf',
        X_train_SIFT, X_test_SIFT,
        y_train_SIFT, y_test_SIFT,
        y_pred_train_SIFT, y_pred_test_SIFT)

y_pred_train_SIFT = knn_SIFT.predict(X_train_SIFT)
y_pred_test_SIFT = knn_SIFT.predict(X_test_SIFT)
toExcel('SIFT', 'KNN','60:40', 'KNN',
        X_train_SIFT, X_test_SIFT, y_train_SIFT,
        y_test_SIFT,y_pred_train_SIFT, y_pred_test_SIFT)

y_pred_train_SIFT = svm_SIFT.predict(X_train_SIFT)
y_pred_test_SIFT = svm_SIFT.predict(X_test_SIFT)
toExcel('SIFT', 'KNN','60:40', 'SVM',
        X_train_SIFT, X_test_SIFT, y_train_SIFT,
        y_test_SIFT,y_pred_train_SIFT, y_pred_test_SIFT)


In [51]:
#create the test input / test output format
#create the train input train output format
testHistogramsList_ORB, testTargetsList_ORB = mapFeatureValsToHistogram(
    testBoVWFeatureVals_ORB, visualWords_ORB, TrainedKmeansModel_ORB)
X_test_ORB = np.array(testHistogramsList_ORB)
y_test_ORB = labelEncoder_ORB.transform(testTargetsList_ORB)
y_pred_train_ORB = clf_ORB.predict(X_train_ORB )
y_pred_test_ORB = clf_ORB.predict(X_test_ORB)

toExcel('ORB', 'KNN', '60:40', 'clf',
        X_train_ORB, X_test_ORB, y_train_ORB,
        y_test_ORB,y_pred_train_ORB, y_pred_test_ORB)

y_pred_train_ORB = knn_ORB.predict(X_train_ORB)
y_pred_test_ORB = knn_ORB.predict(X_test_ORB)
toExcel('ORB', 'KNN', '60:40', 'KNN',
        X_train_ORB, X_test_ORB, y_train_ORB,
        y_test_ORB,y_pred_train_ORB, y_pred_test_ORB)

y_pred_train_ORB = svm_ORB.predict(X_train_ORB)
y_pred_test_ORB = svm_ORB.predict(X_test_ORB)
toExcel('ORB', 'KNN', '60:40', 'SVM',
        X_train_ORB, X_test_ORB, y_train_ORB,
        y_test_ORB,y_pred_train_ORB, y_pred_test_ORB)

In [52]:
#create the test input / test output format
#create the train input train output format
testHistogramsList_SURF, testTargetsList_SURF = mapFeatureValsToHistogram(
    testBoVWFeatureVals_SURF, visualWords_SURF, TrainedKmeansModel_SURF)
X_test_SURF = np.array(testHistogramsList_SURF)
y_test_SURF = labelEncoder_SURF.transform(testTargetsList_SURF)
y_pred_train_SURF = clf_SURF.predict(X_train_SURF )
y_pred_test_SURF = clf_SURF.predict(X_test_SURF)

toExcel('SURF', 'KNN', '60:40', 'clf',
        X_train_SURF, X_test_SURF, y_train_SURF,
        y_test_SURF,y_pred_train_SURF, y_pred_test_SURF)


y_pred_train_SURF = knn_SURF.predict(X_train_SURF)
y_pred_test_SURF = knn_SURF.predict(X_test_SURF)
toExcel('SURF', 'KNN', '60:40', 'KNN',
         X_train_SURF, X_test_SURF, y_train_SURF,
        y_test_SURF,y_pred_train_SURF, y_pred_test_SURF)

y_pred_train_ORB = svm_SURF.predict(X_train_SURF)
y_pred_test_ORB = svm_SURF.predict(X_test_SURF)
toExcel('SURF', 'KNN', '60:40', 'SVM',
        X_train_SURF, X_test_SURF, y_train_SURF,
        y_test_SURF,y_pred_train_SURF, y_pred_test_SURF)

Mean shift

In [53]:
#create the test input / test output format
#create the train input train output format
testHistogramsList_SIFT_ms, testTargetsList_SIFT_ms = mapFeatureValsToHistogram(
    testBoVWFeatureVals_SIFT, visualWords_SIFT_ms, TrainedMsModel_SIFT)
X_test_SIFT_ms = np.array(testHistogramsList_SIFT_ms)
y_test_SIFT_ms = labelEncoder_SIFT_ms.transform(testTargetsList_SIFT_ms)
y_pred_train_SIFT_ms = clf_SIFT_ms.predict(X_train_SIFT_ms )
y_pred_test_SIFT_ms = clf_SIFT_ms.predict(X_test_SIFT_ms)
toExcel('SURF', 'MeanShift', '60:40', 'clf',
        X_train_SIFT_ms, X_test_SIFT_ms, y_train_SIFT_ms,
        y_test_SIFT_ms,y_pred_train_SIFT_ms, y_pred_test_SIFT_ms)


y_pred_train_SIFT_ms = knn_SIFT_ms.predict(X_train_SIFT_ms)
y_pred_test_SIFT_ms = knn_SIFT_ms.predict(X_test_SIFT_ms)
toExcel('SURF', 'MeanShift', '60:40', 'KNN',
        X_train_SIFT_ms, X_test_SIFT_ms, y_train_SIFT_ms,
        y_test_SIFT_ms,y_pred_train_SIFT_ms, y_pred_test_SIFT_ms)

y_pred_train_SIFT_ms = svm_SIFT_ms.predict(X_train_SIFT_ms)
y_pred_test_SIFT_ms = svm_SIFT_ms.predict(X_test_SIFT_ms)
toExcel('SURF', 'MeanShift', '60:40', 'CVM',
        X_train_SIFT_ms, X_test_SIFT_ms, y_train_SIFT_ms,
        y_test_SIFT_ms,y_pred_train_SIFT_ms, y_pred_test_SIFT_ms)

In [54]:
#create the test input / test output format
#create the train input train output format
testHistogramsList_ORB_ms, testTargetsList_ORB_ms = mapFeatureValsToHistogram(
    testBoVWFeatureVals_ORB, visualWords_ORB_ms, TrainedMsModel_ORB)
X_test_ORB_ms = np.array(testHistogramsList_ORB_ms)
y_test_ORB_ms = labelEncoder_ORB_ms.transform(testTargetsList_ORB_ms)
y_pred_train_ORB_ms = clf_ORB_ms.predict(X_train_ORB_ms )
y_pred_test_ORB_ms = clf_ORB_ms.predict(X_test_ORB_ms)
toExcel('ORB', 'MeanShift', '60:40', 'clf',
        X_train_ORB_ms, X_test_ORB_ms, y_train_ORB_ms,
        y_test_ORB_ms,y_pred_train_ORB_ms, y_pred_test_ORB_ms)

y_pred_train_ORB_ms = knn_ORB_ms.predict(X_train_ORB_ms)
y_pred_test_ORB_ms = knn_ORB_ms.predict(X_test_ORB_ms)
toExcel('ORB', 'MeanShift', '60:40', 'KNN',
        X_train_ORB_ms, X_test_ORB_ms, y_train_ORB_ms,
        y_test_ORB_ms,y_pred_train_ORB_ms, y_pred_test_ORB_ms)

y_pred_train_ORB_ms = svm_ORB_ms.predict(X_train_ORB_ms)
y_pred_test_ORB_ms = svm_ORB_ms.predict(X_test_ORB_ms)
toExcel('ORB', 'MeanShift', '60:40', 'SVM',
        X_train_ORB_ms, X_test_ORB_ms, y_train_ORB_ms,
        y_test_ORB_ms,y_pred_train_ORB_ms, y_pred_test_ORB_ms)

In [55]:
#create the test input / test output format
#create the train input train output format
testHistogramsList_SURF_ms, testTargetsList_SURF_ms = mapFeatureValsToHistogram(
    testBoVWFeatureVals_SURF, visualWords_SURF_ms, TrainedMsModel_SURF)
X_test_SURF_ms = np.array(testHistogramsList_SURF_ms)
y_test_SURF_ms = labelEncoder_SURF_ms.transform(testTargetsList_SURF_ms)
y_pred_train_SURF_ms = clf_SURF_ms.predict(X_train_SURF_ms )
y_pred_test_SURF_ms = clf_SURF_ms.predict(X_test_SURF_ms)
toExcel('SURF', 'MeanShift', '60:40', 'clf',
        X_train_SURF_ms, X_test_SURF_ms, y_train_SURF_ms,
        y_test_SURF_ms,y_pred_train_SURF_ms, y_pred_test_SURF_ms)

y_pred_train_SURF_ms = knn_SURF_ms.predict(X_train_SURF_ms )
y_pred_test_SURF_ms = knn_SURF_ms.predict(X_test_SURF_ms)
toExcel('SURF', 'MeanShift', '60:40', 'KNN',
        X_train_SURF_ms, X_test_SURF_ms, y_train_SURF_ms,
        y_test_SURF_ms,y_pred_train_SURF_ms, y_pred_test_SURF_ms)

y_pred_train_SURF_ms = svm_SURF_ms.predict(X_train_SURF_ms )
y_pred_test_SURF_ms = svm_SURF_ms.predict(X_test_SURF_ms)
toExcel('SURF', 'MeanShift', '60:40', 'SVM',
        X_train_SURF_ms, X_test_SURF_ms, y_train_SURF_ms,
        y_test_SURF_ms,y_pred_train_SURF_ms, y_pred_test_SURF_ms)

In [56]:
results_DF

,FeatureExtraction,Clustering Detection,Train Data ratio,Classifier Used,Accuracy(tr),Precision(tr),Recal(tr),F1 score (tr),Accuracy (te),Precision (te),Recal(te),F1 score (te)
0,SIFT,KNN,60:40,clf,1.000000,1.000000,1.000000,1.000000,0.5000,0.500000,0.5000,0.496779
1,SIFT,KNN,60:40,KNN,0.571429,0.577778,0.571429,0.562500,0.4725,0.472063,0.4725,0.470431
2,SIFT,KNN,60:40,SVM,0.785714,0.811111,0.785714,0.781250,0.4650,0.441967,0.4650,0.406067
3,ORB,KNN,60:40,clf,1.000000,1.000000,1.000000,1.000000,0.5525,0.558700,0.5525,0.540363
4,ORB,KNN,60:40,KNN,0.821429,0.823077,0.821429,0.821201,0.5600,0.560024,0.5600,0.559956
5,ORB,KNN,60:40,SVM,0.892857,0.911765,0.892857,0.891613,0.5500,0.550080,0.5500,0.549820
6,SURF,KNN,60:40,clf,1.000000,1.000000,1.000000,1.000000,0.4950,0.494103,0.4950,0.475038
7,SURF,KNN,60:40,KNN,0.785714,0.791667,0.785714,0.784615,0.4550,0.451172,0.4550,0.444104
8,SURF,KNN,60:40,SVM,0.785714,0.791667,0.785714,0.784615,0.4550,0.451172,0.4550,0.444104
9,SURF,MeanShift,60:40,clf,1.000000,1.000000,1.000000,1.000000,0.5200,0.521978,0.5200,0.508951


In [58]:
results_DF.to_excel('10_200Results.xlsx')